# Situering
- De eigenaar van een huishoudelijke zonnepaneelinstallatie zou graag continu een voorspelling hebben van de opbrengst van zijn panelen gedurende de komende uren om het eigen verbruik te optimaliseren: bij een verwachte hoge opbrengst kan hij dan bijv. beslissen om de wasmachine aan te zetten. 
- Hij beschikt over de meterstand per uur sedert ongeveer één jaar (solar.csv). 
- Daarnaast zijn ook de gegevens van de waarnemingen van het weer (weather.csv) en 
- de uren van zonsopgang en –ondergang in dezelfde periode periode (sunrise-sunset.xlsx). 

# Vraag
- Stel een regressiemodel op om de opbrengst per uur te voorspellen. 
- Kies een optimaal regressiemodel door verschillende modellen uit te proberen en te vergelijken volgens de "best practices". 
- Kies als maatstaf de gemiddelde afwijking van de absolute waarde op uurbasis.




In [ ]:
import sys
import sklearn
import numpy as np
import pandas as pd
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime

## Preparation and cleanup of solar data

In [ ]:
def prepare_solar():
    solar = pd.read_csv('datasets/solar.csv')  
    print(solar.head(10))
    print('-------------')
    print(solar.tail(10))
    solar['datetime'] = pd.to_datetime(solar['timestamp'].str[0:19],format='%Y-%m-%d %H:%M:%S')      
    solar['datetime_tz'] = solar['datetime'].dt.tz_localize('Europe/Berlin',ambiguous='NaT')
    solar['datetime_utc'] = solar['datetime_tz'].dt.tz_convert('UTC')
    solar['date'] = solar['datetime_utc'].dt.date
    solar['hour'] = solar['datetime_utc'].dt.hour
    solar = solar[['date','hour','kwh']]
    solar.dropna(inplace=True)
    solar.drop_duplicates(inplace=True)
    # todo timediff en alles > 1u er uit. 
    return solar

solar = prepare_solar()
solar.info()
solar.head()

In [ ]:
solar.tail()

## Preparation and cleanup of weather data

In [ ]:
def prepare_weather():
    weather = pd.read_csv("datasets/weather.csv")
    print(weather.head(10))
    weather = weather[['timestamp','temp','pressure','cloudiness','humidity_relative']]
    weather = weather.groupby(by=['timestamp']).mean().reset_index()
    weather['datetime'] = pd.to_datetime(weather['timestamp'],format='%Y-%m-%dT%H:%M:%S')
    weather['date'] = weather['datetime'].dt.date
    weather['hour'] = weather['datetime'].dt.hour   
    weather = weather[['date','hour','temp','pressure','cloudiness','humidity_relative']]
    weather.drop_duplicates(inplace=True)
    return weather

weather = prepare_weather()
weather.info()
weather.head(30)


## Preparation and cleanup of sunrise/sunset data

In [ ]:
%pip install openpyxl

In [ ]:
def prepare_sunrise_sunset():
    sunrise_sunset = pd.read_excel("datasets/sunrise-sunset.xlsx") 
    print(sunrise_sunset.head(10))
    sunrise_sunset.rename(columns={'datum':'date'}, inplace=True)
    sunrise_sunset['sunrise'] = [pd.Timestamp.combine(d,t) for d,t in zip(sunrise_sunset['date'],sunrise_sunset['Opkomst'])]
    sunrise_sunset['noon'] = [pd.Timestamp.combine(d,t) for d,t in zip(sunrise_sunset['date'],sunrise_sunset['Op ware middag'])]
    sunrise_sunset['sunset'] = [pd.Timestamp.combine(d,t) for d,t in zip(sunrise_sunset['date'],sunrise_sunset['Ondergang'])]
    sunrise_sunset['sunrise'] = sunrise_sunset['sunrise'].dt.tz_localize('Europe/Berlin',ambiguous='NaT')
    sunrise_sunset['sunrise'] = sunrise_sunset['sunrise'].dt.tz_convert('UTC')
    sunrise_sunset['sunset'] = sunrise_sunset['sunset'].dt.tz_localize('Europe/Berlin',ambiguous='NaT')
    sunrise_sunset['sunset'] = sunrise_sunset['sunset'].dt.tz_convert('UTC')
    sunrise_sunset['date'] = sunrise_sunset['date'].dt.date
    sunrise_sunset = sunrise_sunset[['date','sunrise','sunset']]
    return sunrise_sunset

sunrise_sunset = prepare_sunrise_sunset()
sunrise_sunset.info()
sunrise_sunset.head()

Combine solar and weather data in a single dataframe. 

In [8]:
solar_weather = pd.merge(solar,weather, on=['date', 'hour'],how='inner')


Now also combine this dataset with sunrise_sunset. 

In [12]:
all = pd.merge(solar_weather, sunrise_sunset,on=['date'],how='inner')

all.tail()

,date,hour,kwh,temp,pressure,cloudiness,humidity_relative,sunrise,sunset
9335,2024-04-19,4.0,2417.3321,8.750,1014.100,7.5,93.900000,2024-04-19 04:39:00+00:00,2024-04-19 18:45:00+00:00
9336,2024-04-19,5.0,2417.3321,9.300,1013.675,7.5,93.600000,2024-04-19 04:39:00+00:00,2024-04-19 18:45:00+00:00
9337,2024-04-19,6.0,2417.3336,9.575,1013.475,8.0,91.666667,2024-04-19 04:39:00+00:00,2024-04-19 18:45:00+00:00
9338,2024-04-19,7.0,2417.3701,9.925,1013.325,8.0,91.066667,2024-04-19 04:39:00+00:00,2024-04-19 18:45:00+00:00
9339,2024-04-19,8.0,2417.4788,10.400,1012.975,8.0,88.466667,2024-04-19 04:39:00+00:00,2024-04-19 18:45:00+00:00


## Feature Engineering
Only keep following features: 
- dayinyear: number of the day in de year (1/1 = 1, 31/12 = 365)
- sunrise_delta: hours after sunrise
- sunset_delta: hours before sunset
- temp
- pressure
- cloudiness
- humidity
- production (kW): yield of the current hour

In [ ]:
input.info()

In [ ]:
input['date'] = pd.to_datetime(input['date'])
input['dayinyear'] = input['date'].dt.dayofyear

input.head()

In [ ]:
# - sunrise_delta: hours after sunrise
# - sunset_delta: hours before sunset
# - temp
# - pressure
# - cloudiness
# - humidity
# - production (kW): yield of the current hourinput.info()

input['sunrise_delta'] = input['hour'] - input['sunrise'].dt.hour 
input['sunset_delta'] = input['hour'] - input['sunset'].dt.hour 
input = input.drop(columns=['date', 'sunrise', 'sunset', 'date_only'])
input.head()

input.

Create a histogram for all numerical features

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.flatten()

input['temp'].hist(ax=axes[0])
axes[0].set_title('Temperature')

input['pressure'].hist(ax=axes[1])
axes[1].set_title('Pressure')

input['cloudiness'].hist(ax=axes[2])
axes[2].set_title('Cloudiness')

input['humidity_relative'].hist(ax=axes[3])
axes[3].set_title('Relative Humidity')

input['kwh'].hist(ax=axes[4])
axes[4].set_title('kWh')

input['sunrise_delta'].hist()

In [ ]:
input.describe()

Which column requires further attention? Find the odd data en fix it. 

Store the current dataframe to a csv file so we can use it later. 

Read the data from the csv file

Split the dataset in a training and a testset

Create a Random Forest model to predict the hourly production. 
- Create a pipeline with a StandardScaler and and a random forest regressor
- Find the optimal parameter combination amongst
  - bootstrap: False, True
  - n_estimators: 50 - 200 with steps of 50
  - max_depth: 10 - 50 with steps of 10 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

attribs = train_set.drop('production', axis=1)
labels = train_set['production'].copy()

forest_reg = RandomForestRegressor(random_state=42)
param_grid = [
    {'randomforestregressor__n_estimators': np.arange(50,201,50), 'randomforestregressor__max_depth': np.arange(10,51,10)},
    {'randomforestregressor__bootstrap': [False], 'randomforestregressor__n_estimators': np.arange(50,201,50), 
              'randomforestregressor__max_depth': np.arange(10,51,10)} 
  ]

pipeline = make_pipeline(StandardScaler(), forest_reg)
grid_search = GridSearchCV(pipeline, param_grid, cv=4, scoring='neg_mean_absolute_error', return_train_score=True,
                           verbose=4, n_jobs=-1)

grid_search.fit(attribs, labels)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Determine the mean absolute error on the test set. Is this a useful model? 

In [ ]:
from sklearn.metrics import mean_absolute_error

final_model = grid_search.best_estimator_
test_attribs = test_set.drop('production', axis=1)
test_labels = test_set['production'].copy()
test_predictions = final_model.predict(test_attribs)
final_mae = mean_absolute_error(test_labels, test_predictions)
final_mae

Explain the concept of noise in this context

beschaduwing, vervuiling

Store the model to a file. 

In [ ]:
import joblib
joblib.dump(final_model, 'solar_production.pkl')